## Derive explanations on 10 instances from CoQA with simple prompting (let's think step by step)

### Observations

- explanation with wrong prediction only one where the model did not compare options
- need more consisten formatting for the output - add langchain output formatter - but do we then lose the authenticity? look into this..
- where / how to store outputs

\
Other:
- Installing directly from the notebook while connected to the BMW server did not work (tried to set proxy with lines below). Ended up installing the required packages from the terminal (works).


In [ ]:
#!source /Users/q616967/.bash_profile
#!echo $HTTP_PROXY

#!pip3 install python-dotenv langchain openai tiktoken

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
_

True

In [2]:
import sys
sys.path.append('../')

In [22]:
from utils.data import load_data, flatten_CoQA

data = flatten_CoQA(load_data(split='dev'))
data = data[30:40]

import pandas as pd

df = pd.DataFrame(data).drop(columns=["id", "question_concept"])
df


,answerKey,choice_A,choice_B,choice_C,choice_D,choice_E,stem
0,B,eat,hasten,antedate,bring forward,advance,If you are prone to postpone work what will yo...
1,A,library,subway station,county engineer's office,super market,home,James wanted to find an old underground map fr...
2,E,take time,dawdle,go slowly,ocean,slow down,"Sean was in a rush to get home, but the light ..."
3,D,have patience,get in line,sing,stand in line,turn left,Where would a person be doing when having to w...
4,E,satisfaction,heart,feel better,pay,happiness,"She was always helping at the senior center, i..."
5,C,keep cloesd,train,ignition switch,drawer,firearm,"The lock kept the steering wheel from moving, ..."
6,C,beat,direct traffic,city,street,president,Who is a police officer likely to work for?
7,B,quandry,refrigerator,oven,night stand,bakery,"If you have leftover cake, where would you put..."
8,A,whirlpool bath,coffee cup,cup,soft drink,puddle,"A human wants to submerge himself in water, wh..."
9,B,facade,front door,doorway,entrance porch,hallway,Where is a doormat likely to be in front of?


In [36]:
data[:2]

[{'answerKey': 'B',
  'id': 'bdcfbe2132295d437e4c5701085f19c0',
  'question_concept': 'postpone',
  'choice_A': 'eat',
  'choice_B': 'hasten',
  'choice_C': 'antedate',
  'choice_D': 'bring forward',
  'choice_E': 'advance',
  'stem': 'If you are prone to postpone work what will you have to do in order to finish on time?'},
 {'answerKey': 'A',
  'id': '8d3dc21a53523850ec80771daaa5ff20',
  'question_concept': 'underground map',
  'choice_A': 'library',
  'choice_B': 'subway station',
  'choice_C': "county engineer's office",
  'choice_D': 'super market',
  'choice_E': 'home',
  'stem': 'James wanted to find an old underground map from the 50s.  Where might he look for one?'}]

In [23]:
questions = {idx:q['stem'] for idx, q in enumerate(data)}
questions

{0: 'If you are prone to postpone work what will you have to do in order to finish on time?',
 1: 'James wanted to find an old underground map from the 50s.  Where might he look for one?',
 2: 'Sean was in a rush to get home, but the light turned yellow and he was forced to do what?',
 3: 'Where would a person be doing when having to wait their turn?',
 4: 'She was always helping at the senior center, it brought her what?',
 5: 'The lock kept the steering wheel from moving, but the thief still took his chances and began to work on the what?',
 6: 'Who is a police officer likely to work for?',
 7: 'If you have leftover cake, where would you put it?',
 8: 'A human wants to submerge himself in water, what should he use?',
 9: 'Where is a doormat likely to be in front of?'}

In [10]:
from langchain import PromptTemplate

template_CoQA = """
Answer the following question. There are 5 possible answer and only 1 is correct.

Question: {question}
- A: {choice_A}
- B: {choice_B}
- C: {choice_C}
- D: {choice_D}
- E: {choice_E}

Choose the correct answer. Let's think step by step.
"""

# question_concept in template?

In [11]:
""" visualise prompts """

prompt = PromptTemplate(
    input_variables=[
        "question",
        "choice_A",
        "choice_B",
        "choice_C",
        "choice_D",
        "choice_E"
    ],
    template=template_CoQA
)

for instance in data:
    print(prompt.format(
    question=instance["stem"],
    choice_A=instance["choice_A"],
    choice_B=instance["choice_B"],
    choice_C=instance["choice_C"],
    choice_D=instance["choice_D"],
    choice_E=instance["choice_E"]
    ))


Answer the following question. There are 5 possible answer and only 1 is correct.

Question: If you are prone to postpone work what will you have to do in order to finish on time?
- A: eat
- B: hasten
- C: antedate
- D: bring forward
- E: advance

Choose the correct answer. Let's think step by step.


Answer the following question. There are 5 possible answer and only 1 is correct.

Question: James wanted to find an old underground map from the 50s.  Where might he look for one?
- A: library
- B: subway station
- C: county engineer's office
- D: super market
- E: home

Choose the correct answer. Let's think step by step.


Answer the following question. There are 5 possible answer and only 1 is correct.

Question: Sean was in a rush to get home, but the light turned yellow and he was forced to do what?
- A: take time
- B: dawdle
- C: go slowly
- D: ocean
- E: slow down

Choose the correct answer. Let's think step by step.


Answer the following question. There are 5 possible answer an

In [12]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0)

In [19]:
from langchain.chains import LLMChain

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template_CoQA),
    verbose=False
)

In [24]:
responses = {}
#responses = []

for idx, instance in enumerate(data):
    response = llm_chain(
        return_only_outputs=True, 
        inputs={
            'question':instance["stem"],
            'choice_A':instance["choice_A"],
            'choice_B':instance["choice_B"],
            'choice_C':instance["choice_C"],
            'choice_D':instance["choice_D"],
            'choice_E':instance["choice_E"]
            }
        )
    responses[idx] = response['text']

    #og_id = instance['id']
    #responses.append({'id':idx, 'og_id':og_id, 'response':response['text']})

In [35]:
import pprint as pp

pp.pprint(responses, width=120)

{0: 'If you are prone to postpone work, it means that you tend to delay or put off doing your work. In order to finish '
    'on time, you would need to do the opposite of postponing, which is to complete the work earlier or ahead of '
    'schedule. \n'
    '\n'
    'Out of the given options, the correct answer would be:\n'
    '\n'
    '- E: advance',
 1: 'To find an old underground map from the 50s, James should first consider places that are likely to have '
    "historical documents or archives. The most logical options would be the library or the county engineer's "
    'office. \n'
    '\n'
    'While a subway station might have some historical information, it is less likely to have an old underground map '
    "from the 50s readily available. Similarly, a supermarket or James' home are unlikely places to find such a "
    'specific and historical document.\n'
    '\n'
    "Therefore, the correct answer is either A: library or C: county engineer's office.",
 2: 'The question sta

In [40]:
# change respnses to list of dicts
# convert to correct output format: id, original id from dataset, generated explanation

responses_list = [{'id':idx, 
                   'og_id': data[idx]['id'], 
                   'response':response
                   } for idx, response in responses.items()]

responses_list[:2]

[{'id': 0,
  'og_id': 'bdcfbe2132295d437e4c5701085f19c0',
  'response': 'If you are prone to postpone work, it means that you tend to delay or put off doing your work. In order to finish on time, you would need to do the opposite of postponing, which is to complete the work earlier or ahead of schedule. \n\nOut of the given options, the correct answer would be:\n\n- E: advance'},
 {'id': 1,
  'og_id': '8d3dc21a53523850ec80771daaa5ff20',
  'response': "To find an old underground map from the 50s, James should first consider places that are likely to have historical documents or archives. The most logical options would be the library or the county engineer's office. \n\nWhile a subway station might have some historical information, it is less likely to have an old underground map from the 50s readily available. Similarly, a supermarket or James' home are unlikely places to find such a specific and historical document.\n\nTherefore, the correct answer is either A: library or C: county eng